In [3]:
import MDAnalysis as mda

In [4]:
u = mda.Universe('/home/kanaka/from_box/trp/pymol_selections/7mij_pml_chains.pdb')

/home/kanaka/.local/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:451: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


In [5]:
segids = set(u.residues.segids)
# hold the coms for each protomer 
seg_sels = {seg: u.select_atoms(f'segid {seg}') for seg in set(u.residues.segids)} # if segid in subunits

In [6]:
# access a subunit com
from itertools import combinations, permutations
triples = [[a,b,c] for a,b,c in permutations(segids, 3)]
coms = {segid: seg_sels[segid].center_of_mass() for segid in segids}


In [7]:
import numpy as np
# triple selections without duplicating angle created by vertex and permutations of the 2 points
trips = []
for segid in segids:
    trips.append([[ch1,segid,ch2] for ch1,ch2 in combinations([ch for ch in segids if ch != segid],2)])

In [8]:
trips

[[['A', 'C', 'D'], ['A', 'C', 'B'], ['D', 'C', 'B']],
 [['C', 'A', 'D'], ['C', 'A', 'B'], ['D', 'A', 'B']],
 [['C', 'D', 'A'], ['C', 'D', 'B'], ['A', 'D', 'B']],
 [['C', 'B', 'A'], ['C', 'B', 'D'], ['A', 'B', 'D']]]

In [9]:
all_seg_sel_string = ''.join([f'segid {seg} or ' for seg in list(segids)[:-1]])
all_seg_sel_string += f'segid {list(segids)[-1]}'
all_com = u.select_atoms(all_seg_sel_string).center_of_mass()

In [10]:
def get_angle(a,b,c):
    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return angle

In [11]:
opposing_subunits = []
check = []
for seg in segids:
    if seg not in check:
        for seg2 in [segid for segid in segids if segid != seg]:
            if np.abs(180-np.rad2deg(get_angle(coms[seg],all_com,coms[seg2]))) <= 2:
                opposing_subunits.append((seg,seg2))
                check.append(seg)
                break




In [12]:
opposing_subunits

[('C', 'B'), ('A', 'D'), ('D', 'A'), ('B', 'C')]

In [80]:
np.rad2deg(get_angle(coms['A'],all_com,coms['D']))

179.92590933219583